In [1]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import random
import numpy as np

In [2]:
distributions = pd.read_csv('/Users/frieda/Desktop/distributions.csv')
cost_matrix = pd.read_csv('/Users/frieda/Desktop/cost_matrix.csv')

In [4]:
# Display descriptive statistics
print("Distribution statistics:")
print(distributions.describe())
print("\nCost matrix statistics:")
print(cost_matrix.describe())

# Model parameters
n = distributions['n']
p = distributions['p']
num_stores = cost_matrix.shape[0]

# Initialize model 
model = gb.Model("Baffin Bay")

# Total objectives and sum of decision variables
total_objectives = 0
total_capacities = [0] * num_stores

# Number of trials
num_trials = 50

# Scenarios per trial
num_scenarios = 100
    
for _ in range(num_trials):

    # Add common capacity decision variables
    y = model.addVars(num_stores, lb=0, vtype=gb.GRB.INTEGER, name="order")

    # Deviational variables in the average model 
    d_plus = model.addVars(num_stores, num_scenarios, lb=0, vtype=gb.GRB.INTEGER, name="excess_inventory")
    d_minus = model.addVars(num_stores, num_scenarios, lb=0, vtype=gb.GRB.INTEGER, name="excess_demand")    
    
    # Deterministic objective function 
    underage_cost = gb.quicksum((24.44) * d_minus[i, n] for i in range(num_stores) for n in range(num_scenarios)) / num_scenarios
    overage_cost = gb.quicksum((25.55) * d_plus[i, n] for i in range(num_stores) for n in range(num_scenarios)) / num_scenarios
    model.setObjective(underage_cost + overage_cost, gb.GRB.MINIMIZE)
    
    # Demand constraint
    for s in range(num_scenarios):
        for i in range(num_stores):
            model.addConstr(y[i] + d_minus[i, s] == np.random.binomial(n[i], p[i]) + d_plus[i, s])

    # Solve the problem optimally
    model.optimize()
    
    # Update running total
    total_objectives += model.objVal
    total_capacities = [total_capacities[i] + y[i].x for i in range(num_stores)]
    
    # Reset the model for the next trial
    model.reset(0)
    
# Output results
print("Average Objective: ", total_objectives / num_trials)
for idx, capacity in enumerate(total_capacities):
    print(f"Average Optimal Capacity at store {idx}: {capacity / num_trials}")

Distribution statistics:
                n          p        Mean
count   15.000000  15.000000   15.000000
mean   116.933333   0.779167   92.022030
std     19.185064   0.129295   23.857183
min     82.000000   0.580407   51.816857
25%    104.000000   0.682420   74.591805
50%    119.000000   0.780886   98.315728
75%    127.000000   0.864052  112.343791
max    147.000000   0.992744  121.847557

Cost matrix statistics:
               0          1          2          3          4          5  \
count  15.000000  15.000000  15.000000  15.000000  15.000000  15.000000   
mean    1.857698   1.581392   1.865505   1.762188   1.752358   1.951037   
std     0.681176   0.545082   0.641201   0.596589   0.642975   0.703752   
min     0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
25%     1.555797   1.356235   1.591919   1.520011   1.563868   1.850283   
50%     1.841806   1.720943   2.092072   1.901807   1.821132   1.903692   
75%     2.208821   1.916440   2.282031   2.124497   2.20